In [28]:
import polars as pl
from datetime import datetime, timedelta
import numpy as np
import random

np.random.seed(42)
random.seed(42)

# Datas para os registros
datas = [datetime(2023, 1, 1) + timedelta(days=i) for i in range(1000)]

# Localizações possíveis e suas coordenadas centrais aproximadas
locais_e_coordenadas = {
    'São Paulo': (-23.550520, -46.633308),
    'Rio de Janeiro': (-22.906847, -43.172896),
    'Belo Horizonte': (-19.921734, -43.945095),
    'Porto Alegre': (-30.034647, -51.217658),
    'Salvador': (-12.977749, -38.501630),
    'Manaus': (-3.119028, -60.021731),
    'Recife': (-8.047562, -34.877015),
    'Curitiba': (-25.428356, -49.273252),
    'Goiânia': (-16.686891, -49.264794),
    'Fortaleza': (-3.717219, -38.543303),
}
produtos = ['Produto A', 'Produto B', 'Produto C', 'Produto D', 'Produto E']

def gerar_dados_com_locacao(n=1000, variacao_km=300, tipo='compra'):
    variacao_grau_lat = variacao_km / 111  # Conversão aproximada de km para graus de latitude
    dados = []
    for _ in range(n):
        produto = random.choice(produtos)
        local_nome, (lat_centro, lon_centro) = random.choice(list(locais_e_coordenadas.items()))
        # Ajuste na variação para longitude, considerando a latitude
        variacao_grau_lon = variacao_km / (111 * np.cos(np.radians(lat_centro)))
        
        latitude = lat_centro + np.random.uniform(-variacao_grau_lat, variacao_grau_lat)
        longitude = lon_centro + np.random.uniform(-variacao_grau_lon, variacao_grau_lon)
        data = random.choice(datas)
        quantidade = np.random.randint(1, 20)
        preco = np.random.uniform(30.0, 400.0)
        
        if tipo == 'venda':
            # Gerar o fator de markup para cada item individualmente
            markup_fator = np.random.uniform(1.2, 1.8)
            preco *= markup_fator
        
        dados.append([data, local_nome, latitude, longitude, preco, quantidade, produto])
    
    colunas = ['Data', 'Local Compra' if tipo == 'compra' else 'Local Venda', 'Latitude', 'Longitude', 'Preço Compra' if tipo == 'compra' else 'Preço Venda', 'Quantidade', 'Produto']
    return pl.DataFrame({colunas[i]: [row[i] for row in dados] for i in range(len(colunas))})

# Gerar dados de compras e vendas
df_compras = gerar_dados_com_locacao(1000, 120, 'compra')
df_vendas = gerar_dados_com_locacao(1000, 120, 'venda')

# Definir caminhos para os arquivos Parquet
caminho_vendas = ''
caminho_compras = ''

# Salvar os dataframes como arquivos parquet
df_vendas.write_parquet(caminho_vendas)
df_compras.write_parquet(caminho_compras)
